# Fully commented code to interact with openAI chatBot

This small code was thought as a self-assignment aimed to help me understand better how to build a WebApp using Flask, connecting it to an API through Python and also play a bit with AI chatBots.

Being this project connected to an OpenAI chatbot I used their Python Library **"openai"** (https://github.com/openai/openai-python) instead of the generical purpose **"requests"** (https://pypi.org/project/requests/). 

#### Simple overview

The project works as this:

- calling the **app.py** file will create a **Flask web server**, using the html (and corresponding css) file as a template. This file will also execute an **API call** to openAI services (keeping it on as long as the server runs)

- on this server's **webpage**, there is a text box with a submit button. Writing a message and submitting it (**javascript interaction**) will trigger the **Flask server's routes**

- these **routes** will format the message from the user input and send it to a **python function** called get_bot_response (contained in the app.py file too)

- the **get_bot_response** will interact with the **chatbot** and return a response, which will take into considerations the conversation history

- the chatbot route of the flask server will take this response and return it to the javascript function, which will show it on the page

#### Directory structure
Below I will provide the commented codes in all its parts:

    1) the python project, ./app.py
    2) the html file, ./template/index.py
    3) the javascript file, ./static/scripts.js
    4) the css file, ./static/style.css
    
The directory structure, as commented later, depends on how flask webapps works, but could be changed manually.

Also, I'm not adding to the gitHub repository my personal API key, which in my case was stored in the file named **./hidden.txt**.

### References
- what is an API: https://en.wikipedia.org/wiki/API
- REST vs SOAP API: https://aws.amazon.com/compare/the-difference-between-soap-rest/#:~:text=SOAP%20and%20REST%20are%20two%20different%20approaches%20to%20API%20design,exchange%20data%20in%20multiple%20formats
- what is Python: https://www.python.org/about/\
- what is Flask: https://flask.palletsprojects.com/en/2.3.x/
- openAI: https://platform.openai.com/
- video which inspired this small project: https://www.youtube.com/watch?v=qkzhSZAwD6A

## Python/Slack file

To launch the webapp development server it's enough to launch the file app.py file from a terminal, with something like "python3 ./app.py". This will show an IP to connect to to interact with the actual webapp.

Writing this IP in the URL bar will lead to the application. Just clicking on the html file would open the webpage without the app server functioning (preventing the user to interact with the bot) and with no graphics settings (the css is connected to the application, not directly to the html).

### Interaction with the BOT (OpenAI API call, app.py - part 1)

Let's start  with the part of the code containing the function which will ask the bot for a response

In [ ]:
# Library needed to connect to openai services
import openai

# Set your OpenAI API key
with open('./hidden.txt') as file:
    openai.api_key = file.read()

# This list will contain the session conversation history
# to let the bot remember what it was talking about before the last answer
conversation_history = []

# Define a function to get the bot's response
# takes as an input a prompt in the form of a string and gives a string or none 
# in case of errors and if the code wasn't able, for any reason, to provide the error message
def get_bot_response(prompt: str) -> str | None:

    #define a text variable with type string or None, with default value None
    text: str | None = None
    
    #the code is strucured in a try/except model: if no error or warning happens, the code will follow the try section
    #but if something goes wrong, it will follow the except section
    try:
        
        #This two variables will give every time a message is sent as an input prompt all the past messages and response 
        #+ the new message 
        conversation = "\n".join([f"{msg['sender']}: {msg['message']}" for msg in conversation_history])
        prompt_with_history = f"{conversation}\nHuman: {prompt}\nAI:"

        #this dictionary will be the bot answer
        response: dict = openai.Completion.create(
            model='text-davinci-003', #the AI model I chose from openAI websites, many are available
            
            prompt=prompt_with_history, #text provided to the model as a starting point, 
                                        #which includes the user prompt and its history
            
            temperature=0.9, #this parameter of the answers the AI gives
                            #goes from 0 (focused and deterministic) to 1 (random and creative)
            
            max_tokens=300, #changes the answer max possible length (also the cost of the API use)
            
            top_p=1, #this sampling parameter tells the model wether to use for its answers 
                     #a lower and more selected (close to 0) or vast (close to 1) number of possible tokens
            
            frequency_penalty=0, #this tells the model whether to generate content using different words (close to 1)
                                 #or to use a more repetitive form (close to 0)            
            
            presence_penalty=0.6, #the same, but here the creativity is linked to the answer compared to the imput
                                  #a higher value makes the model answer without following strictly the question form
            
            stop=[' Human:', ' AI:'] #a set of strings which the model should interpret as a stop to generating more content
        )
        
        #to see the json structure of the dictionary uncomment the following
        #print(response) 

        #we save in a new dictionary the choiches dictionary, which is part of the response object
        choices: dict =  response.get('choices')[0]
        
        #uncomment to see the actual structure of the dictionary
        #print(choices) 
        
        #In particular we are interested in the text voice of choices, which is our answer
        text = choices.get('text')

    # Print an error message in case the bot cannot respond properly for any reason
    except Exception as e:
        print('ERROR:', e)
        text = f'Something went wrong: {e}'

    return text

#### Testing the get_bot_response function
Here is shown a small test use of the get_bot_response function, which will give as an output a string (the answer from response.get('choices')[0].get('text'))

As you can see, you can get freaky with the input prompt

In [ ]:
prompt = "You are an intelligent chicken like alien coming for the fist time on Earth, give me you impressions from your POV"
get_bot_response(prompt)

### Flask WebApp (app.py - part 2)
This part of the app.py file will implement a **Web application**.

This code references the **page layout** as defined in the html template (and the css file linked to that), but it also defines **routes** to manage the input/output of the user message and the bot message to be handled by the page.

The javascript file will show on the page the answer returned by the chatbot call, triggered by the user submitting a message on the page, which is the trigger to the chatbot route and then of the API call.

In this context the prompt will be referred to as "message", to separate it from the bot's "response"

In [ ]:
# Libraries needed to create a flask webapplication
from flask import Flask, render_template, request

#create a Flask application instance called app
# '__name__' indicate the Python's module which is being used
app = Flask(__name__)

#Define a home route at the URL '/' (home page)
#When accessing this route, the home function is called, which render the HTML "index.html'
#saved in the ./template directory
@app.route('/')
def home():
    return render_template('index.html')

#Define a route to handle requests to the chatbot endpoint
#It only accept POST requests
#when accessing this route the chatbot function is call
@app.route('/chatbot', methods=['POST'])
def chatbot():
    # Get the user's message from the form data
    message = request.form['message']

    # Format the user's message as an HTML string
    user_html = f'<p><strong>You:</strong> {message}</p>'

    # Format the message as a prompt for the bot
    prompt = f'\nHuman: {message}\nAI:'

    # Get the bot's response using the get_bot_response function from above
    response = get_bot_response(prompt)

    # Format the bot's response as an HTML string
    # ndr: the AI: format is required for the API call
    # but I chose to use the friendlier "bot" for the actual webapp
    bot_html = f'<p><strong>Bot:</strong> {response}</p>'

    # Update conversation history list
    conversation_history.append({'sender': 'user', 'message': message})
    conversation_history.append({'sender': 'bot', 'message': response})

    # Return both messages as HTML strings
    return user_html + bot_html

#These if grants that the Flask App can run only when the app.py script is executed directly
#(and not if called as a module, for example)
#it's call is in the js file
if __name__ == '__main__':
    #debug=True enables debugging mode, not necessary but useful while developing
    app.run(debug=True)

## Web app source code
In this section will be shown the codes used for the actual webapp GUI.

As they are very simple and most of the graphical details are arbitrary and not really necessary I will comment only on the critical parts.

### HTML

In [ ]:
<!DOCTYPE html>
<html>

  <head>
    <title>OpenAI Chatbot</title>
    
    #Referral to the css file in the ./static/ directory
    <link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='style.css') }}">
  </head>

  <body>
    #The container div will cover the whole page
    <div class="container">
    
      <header>
        <h1>OpenAI Chatbot</h1>
      </header>
    
      #Input form
      <form id="chat-input">
        <textarea name="message" id="message" placeholder="Type your message..." rows="3"></textarea>
        <button type="submit">Send</button>
      </form>
    
      #Chat history div, filled from the js functions
      <main id="chat"></main>
    </div>
    
    #jquery is included to simplify the dynamic DOM manipulation while updating the page
    #while doing AJAX requests
    #It also helps handling the form submission event (to trigger the bot response)
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    
    #These js functions handles the the display of user/bot interactions and manage the chat interface behaciour
    <script src="{{ url_for('static', filename='scripts.js') }}"></script>
  </body>
</html>

### JavaScript

In [ ]:
#This function handle the message div, if the ENTER button is pressed, send the message
#if ENTER+shift is pressed, it add a line break to the message
$('#message').keydown(function(event) {
    if (event.keyCode == 13 && event.shiftKey) {
        var content = this.value;
        var caret = getCaret(this);
        this.value = content.substring(0,caret)+"\n"+content.substring(caret,content.length-1);
        event.stopPropagation();
        } else if (event.keyCode == 13) {
            $('#chat-input').submit();
            #if only ENTER is pressed, submit ignoring the default "line break" behaviour of the key
            event.preventDefault();
            }
    });

#This function retrieve the caret (cursor) position within a given 'el' element
#and return the index of the caret position
#it is used in the previous function to know the end of the message
function getCaret(el) {
    if (el.selectionStart) {
        return el.selectionStart;
        } else if (document.selection) {
            el.focus();
            var r = document.selection.createRange();
            if (r == null) {
                return 0;
                }
            var re = el.createTextRange(), rc = re.duplicate();
            re.moveToBookmark(r.getBookmark());
            rc.setEndPoint('EndToStart', re);
            return rc.text.length;
            }
    return 0;
    }

#triggers when the submit button (or event) are triggered, making an AJAX POST request
#to the '/chatbot' endpoint of the server
#this is the connection between user interaction with the front-end of the app and the API call
$('form').submit(function(event) {
    #prevent the default form behaviour of refreshing the page
    event.preventDefault();

    #send the AJAX request to the chatbot side
    $.post('/chatbot', $(this).serialize(), function(response) {

        #append the response to the #chat div
        $('#chat').append(response);

        #clear the input form to let the user write again
        $('#message').val('').focus();
        });
    });

## CSS

In [ ]:
body {
    font-family: Arial, sans-serif;
    background-color: #f7f7f7;
    margin: 0;
  }
  
  header {
    background-color: #f9d71c;
    color: #393939;
    padding: 1rem;
    text-align: center;
  }
  
  h1 {
    margin: 0;
  }
  
  .container {
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: center;
    width: 50vw;
    background-color: lightyellow;
    margin: 0 auto; /* Center horizontally */
    /* max-height: 80vh; Set a max height */
    overflow-y: auto; /* Add vertical scrollbar when content overflows */
  }
  
  main {
    flex-grow: 1;
    display: flex;
    flex-direction: column;
    justify-content: flex-end;
    padding: 1rem;
    background-color: lightyellow;
    max-width: 80%; /* Set maximum width */
    margin: auto; /* Center horizontally */
    overflow-y: auto; /* Add this line to enable scrolling */
  }
  
  .message {
    margin: 0.5rem 0;
    max-width: 100%; /* Set maximum width */
    word-wrap: break-word; /* Enable word wrapping */
    /* max-height: 50px; Set a max height for each message */
    overflow-y: auto; /* Add vertical scrollbar when content overflows */
  }
  
  .user-message {
    text-align: right;
  }
  
  textarea {
    font-size: 1.2rem;
    padding: 1rem;
    border: none;
    border-radius: 0;
    border-bottom: 2px solid #ccc;
    width: 80%;
    height: 6rem;
    overflow-y: auto;
    background-color: #fff8dc;
    color: #393939;
    text-align: center;
    margin-top: 2rem;
    margin-bottom: 2rem;
  }
  
  button[type="submit"] {
    font-size: 1.2rem;
    padding: 0.5rem;
    background-color: #f9d71c;
    color: #393939;
    border: none;
    border-radius: 0;
    cursor: pointer;
  }